In [29]:
import os
import cv2
import torch
from torchvision import transforms
from torch.nn.functional import cosine_similarity
from skimage.metrics import structural_similarity as ssim
import numpy as np
from skimage.transform import resize
import pandas as pd

train_chart_folder = "train/chart"
train_nonchart_folder = "train/nonchart"

to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)) 
])

def compute_similarity(input_tensor, train_tensors):
    cos_sims = []
    ssim_scores = []
    input_np = input_tensor.permute(1,2,0).numpy()

    for t in train_tensors:
        cos_sim = cosine_similarity(
            input_tensor.flatten().unsqueeze(0),
            t.flatten().unsqueeze(0)
        ).item()
        cos_sims.append(cos_sim)

        t_np = t.permute(1,2,0).numpy()
        if t_np.shape != input_np.shape:
            t_np_resized = resize(t_np, input_np.shape, preserve_range=True, anti_aliasing=True)
        else:
            t_np_resized = t_np
        ssim_score, _ = ssim(t_np_resized, input_np, channel_axis=2, full=True, data_range=1.0)
        ssim_scores.append(ssim_score)

    return cos_sims, ssim_scores

def panda_for_analytics(folder, input_tensor, cos_sims, ssim_scores, excel_name):
    rows = []
    
    image_files = [f for f in os.listdir(folder) if f.endswith((".png", ".jpg", ".jpeg"))]
    image_files = image_files[:len(cos_sims)]
    
    for idx, filename in enumerate(image_files):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        t = to_tensor(img)
        
        cos_sim = cos_sims[idx]
        ssim_score = ssim_scores[idx]
        
        row = {
            "Folder": folder,
            "File": filename,
            "inputimagename": "page.png", 
            "Shape": tuple(t.shape),
            "Min": t.min().item(),
            "Max": t.max().item(),
            "Mean": t.mean().item(),
            "cosinesim": cos_sim,
            "ssim_scores": ssim_score
        }
        rows.append(row)

    df = pd.DataFrame(rows)
    df.to_excel(excel_name, index=False, engine='openpyxl')
    return df



In [30]:
def preprocess_input_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    tensor = to_tensor(img)
    return tensor

input_tensor = preprocess_input_image("page.png")


In [31]:
def load_images_as_tensors(folder):
    tensors = []
    for filename in os.listdir(folder):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            tensor = to_tensor(img)
            tensors.append(tensor)
    return tensors

chart_tensors = load_images_as_tensors(train_chart_folder)
nonchart_tensors = load_images_as_tensors(train_nonchart_folder)

In [32]:
cos_chart, ssim_chart = compute_similarity(input_tensor, chart_tensors)
cos_nonchart, ssim_nonchart = compute_similarity(input_tensor, nonchart_tensors)

print("Max Cosine with chart images:", max(cos_chart))
print("Max Cosine with non-chart images:", max(cos_nonchart))
print("Max SSIM with chart images:", max(ssim_chart))
print("Max SSIM with non-chart images:", max(ssim_nonchart))

chart_df = panda_for_analytics(train_chart_folder, input_tensor, cos_chart, ssim_chart, "chart_stats.xlsx")
# nonchart_df = panda_for_analytics(train_nonchart_folder, input_tensor, cos_nonchart, ssim_nonchart, "nonchart_stats2.xlsx")


Max Cosine with chart images: 0.9999929070472717
Max Cosine with non-chart images: 0.9954390525817871
Max SSIM with chart images: 0.9975357
Max SSIM with non-chart images: 0.5917752
